In [ ]:
import numpy as np
import pyvista as pv
from tqdm import tqdm

from kale import Engine, save_movie, show_ui
from kale.algorithms import (
    cell_data_to_point_data_algorithm,
    contour_banded,
    extract_feature_edges_algorithm,
    scalars_operation_algorithm,
    subdivide_algorithm,
    warp_by_scalar_algorithm,
)
from kale import theme

In [ ]:
# Theming adjustments
pv.global_theme.image_scale = 5

# Load Data

In [ ]:
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.01

In [ ]:
engine = Engine(MESH_GEOMETRY_FILE_NAME, MESH_VALUES_FILE_NAME, zscale=MESH_Z_SCALE)
engine.keys

# Find Camera Position

This is a helper section to find your preferred camera location

In [ ]:
plc = pv.Plotter()
plc.add_mesh(engine.mesh, color="beige")
plc.show()

In [ ]:
# copy value of this output into next cell after moving scene above
plc.camera_position

In [ ]:
# set the camera position (different for different datasets)
cpos = [
    (117.91283696478234, 37.24460195609967, 6.366970274654328),
    (134.58767110519122, 32.70077461447109, -0.3100684512554164),
    (0.3427373584560485, -0.1127261631888837, 0.9326435091991383),
]

# Choosing data array

The following cell is where you can control which variable is being plotted:

In [ ]:
engine.mesh.active_scalars_name = "cumulative_slip"

In [ ]:
# Add a surface warping algorithm
warp = warp_by_scalar_algorithm(
    cell_data_to_point_data_algorithm(engine.algorithm),
    scalars="geometric_moment",
    factor=0.2e-8,  # This factor is incredibly data-dependent
)
boundary = extract_feature_edges_algorithm(
    warp,
    boundary_edges=True,
    non_manifold_edges=False,
    feature_edges=False,
    manifold_edges=False,
)

# Visualize

This cell will not produce an output, but instead renders off-screen to create screenshots.

In [ ]:
# Set contour levels
# CONTOUR_LEVELS = np.linspace(-10e9, 10e9, 11)
# CONTOUR_LEVELS = np.linspace(-2500, 2500, 11)
CONTOUR_LEVELS = np.linspace(-5, 15, 11)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

contour, edges = contour_banded(
    warp,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=engine.mesh.active_scalars_name,
)

In [ ]:
annotations = {
    np.sum(CLIM) / 2: "Cumulative Slip",
}

#### Plotting code ####
pl = pv.Plotter()
pl.add_mesh(
    contour,
    cmap=theme.COLOR_MAPS[engine.mesh.active_scalars_name],
    clim=CLIM,
    scalars=engine.mesh.active_scalars_name,
    n_colors=N_COLORS,
    show_scalar_bar=True,
    scalar_bar_args=dict(
        title="",
        vertical=False,
        height=0.05,
        width=0.33,
        position_x=0.33,
        position_y=0.1,
        **theme.SCALAR_BAR_OPTS,
    ),
    annotations=annotations,
)
pl.add_mesh(edges)

pl.add_mesh(boundary, color="black")

pl.add_floor(
    "-z",
    show_edges=True,
    edge_color="white",
    color="lightgray",
)

# Use camera position set earlier
pl.camera_position = cpos
# pl.camera.zoom(1.5)

# Drawing lines as a grid proxy
# mesh = pv.Line((0, 0, 0), (0, 0, 1))
# mesh.plot(color='k', line_width=10)

# Experimenting with lights
light = pv.Light((-2, 2, 0), (0, 0, 0), "white")
pl.add_light(light)


# Make a video over time steps
step = 100
pl.open_movie(f"warp-{engine.mesh.active_scalars_name}.mp4", framerate=30, quality=5)
for tindex in tqdm(range(1, engine.max_time_step, step)):
    engine.time_step = tindex
    pl.add_text(
        f"t = {tindex:05d}",
        name="time-step-label",
        font_size=theme.AXES_FONT_SIZE * 2,
        position="upper_edge",
    )
    pl.write_frame()
pl.close()

# show_ui(engine, pl, continuous_update=False)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with 2 rows and 3 columns
fig, axes = plt.subplots(2, 3, figsize=(12, 4))

# Flatten the axes array to iterate over it easily
axes = axes.flatten()

# Iterate over the data and plot histograms
for i, tstep in enumerate(np.linspace(0, engine.max_time_step - 1, num=6, dtype=int)):
    # Select the current axis
    ax = axes[i]

    engine.time_step = int(tstep)
    # Plot the histogram
    ax.hist(engine.mesh["geometric_moment"], bins=10)

    # Set title and labels
    ax.set_title(f"Histogram t={tstep}")
    ax.set_xlabel("geometric_moment")
    ax.set_ylabel("Frequency")

    ax.axvline(x=0, color="r", linestyle="--")

# Adjust the spacing between subplots
plt.tight_layout()

# Display the plots
plt.show()